In [1]:
useMock=False

In [2]:
from cffi import FFI
ffi = FFI()
with open('extracted_headers/safe_c_ffi_data.h','r') as f:
    myHeaders=f.read()
ffi.cdef(myHeaders)
with open('extracted_headers/safe_c_ffi_funcs.h','r') as f:
    funs=f.read()
ffi.cdef(funs)
lib=ffi.dlopen('../compiled_binaries/libsafe_app' + useMock*'_mock' + '.so')

---

maybe it makes most sense to really define the data types by hand and just check them for changes with updates on the api - not entirely sure

- rid

---

```c
typedef unsigned long int uintptr_t;

typedef struct {
 char* id;
 char* scope;
 char* name;
 char* vendor;
} AppExchangeInfo;
```

In [3]:
class AppExchangeInfo:
    def __init__(self,id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
        self.id = ffi.new('char[]',id)
        self.scope = ffi.new('char[]',scope)
        self.name = ffi.new('char[]',name)
        self.vendor = ffi.new('char[]',vendor)

        self.entity = ffi.new('AppExchangeInfo *',[self.id,self.scope,self.name,self.vendor])

In [4]:
def AppExchangeInfo(id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
    id = ffi.new('char[]',id)
    scope = ffi.new('char[]',scope)
    name = ffi.new('char[]',name)
    vendor = ffi.new('char[]',vendor)

    myStruct = ffi.new('AppExchangeInfo *',[id,scope,name,vendor])
    
    return myStruct, [id, scope, name, vendor]

newExChangeInfo,infopayload = AppExchangeInfo(b'0.0.2',name=b'pyTest',vendor=b'rid')

newExChangeInfo,infopayload = AppExchangeInfo(id=b'meh',scope=b'myScope',
                                              name=b'PyTests only today',vendor=b'rid')

ffi.string(newExChangeInfo.vendor)

ffi.string(newExChangeInfo.name)

```c
typedef struct {
 _Bool read;
 _Bool insert;
 _Bool update;
 _Bool delete;
 _Bool manage_permissions;
} PermissionSet;
```

In [5]:
def PermissionSet(read=True,insert=True,update=True,delete=True,manage_permissions=True):
    return ffi.new('PermissionSet *',[read,insert,update,delete,manage_permissions])

newPermissionSet = PermissionSet()

```c
typedef struct {
 char* cont_name;
 PermissionSet access;
} ContainerPermissions;
```

In [6]:
def ContainerPermissions(name=b'noName',access=None):
    containerName = ffi.new('char[]',name)
    if not access:
        access = PermissionSet()
    container = ffi.new('ContainerPermissions *',[containerName,access[0]])
    
    return container, [containerName,access]

newPermissions,permission_addon=ContainerPermissions(b'_pictures',newPermissionSet)

In [7]:
newPermissions,permission_addon=ContainerPermissions(b'_publicNames')

In [8]:
newPermissions,permission_addon=ContainerPermissions(b'_music')

In [9]:
ffi.string(newPermissions.cont_name)

b'_music'

```c
typedef struct {
 AppExchangeInfo app;
 _Bool app_container;
 ContainerPermissions* containers;
 uintptr_t containers_len;
 uintptr_t containers_cap;
} AuthReq;
```

In [10]:
def AuthReq(permissions,containers_len,containers_cap,id=b'noId',scope=b'pythonscript',
            name=b'noName',vendor=b'nobody',app_container=True):
    
    newExChangeInfo,infopayload = AppExchangeInfo(id,scope,name,vendor)
    
    authReq = ffi.new('AuthReq *',[newExChangeInfo[0],app_container,permissions,containers_len,containers_cap])
    
    return authReq, [newExChangeInfo,infopayload]

myAuth,addData=AuthReq(newPermissions,0,0,id=b'ridsTest',scope=b'pythonscript'
                       ,name=b'PyTest',vendor=b'rid never work',app_container=True)

myAuth,addData=AuthReq(newPermissions,0,0,id=b'ridsTest',scope=b'pythonscript'
                       ,name=b'PyTest',vendor=b'nobody',app_container=True)

In [11]:
myAuth,addData=AuthReq(newPermissions,1,1,id=b'newProgId_unique_thing',scope=b'lalali'
                       ,name=b'newProgramName',vendor=b'nobody',app_container=True)

In [12]:
@ffi.callback("void(void *, FfiResult *)")
def feedback(userData, result):
    print(userData)
    print(result.error_code)

In [13]:
lib.app_set_additional_search_path(ffi.new('char[]',b'../compiled_binaries/'),ffi.NULL,feedback)

<cdata 'void *' NULL>
0


@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, noClueWhatNumber, resultString):
    print(userData)
    print(result)
    print(noClueWhatNumber)
    print(resultString)

lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

myAuth.containers.access.insert=True

myAuth.containers.access.delete=True

myAuth.containers.access.read=True

In [14]:
ffi.string(myAuth.app.name)

b'newProgramName'

In [15]:
ffi.string(myAuth.app.id)

b'newProgId_unique_thing'

In [16]:
encodedAuth=None

In [17]:
@ffi.callback("void(void *, FfiResult *, unsigned long int, char *)")
def o_cb(userData, result, reqId, encodedReq):
    global encodedAuth
    print(userData)
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(reqId)
    print(ffi.string(encodedReq))
    encodedAuth = ffi.string(encodedReq)

In [18]:
@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, reqId, encodedReq):
    global encodedAuth
    print(userData)
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(reqId)
    print(ffi.string(encodedReq))
    encodedAuth = ffi.string(encodedReq)

In [19]:
lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

<cdata 'void *' NULL>
0
2681354838
b'AAAAAFY60p8AAAAAFgAAAAAAAABuZXdQcm9nSWRfdW5pcXVlX3RoaW5nAQYAAAAAAAAAbGFsYWxpDgAAAAAAAABuZXdQcm9ncmFtTmFtZQYAAAAAAAAAbm9ib2R5AQEAAAAAAAAABgAAAAAAAABfbXVzaWMFAAAAAAAAAAAAAAABAAAAAgAAAAMAAAAEAAAA'


In [20]:
b'safe-auth://' + encodedAuth

b'safe-auth://AAAAAFY60p8AAAAAFgAAAAAAAABuZXdQcm9nSWRfdW5pcXVlX3RoaW5nAQYAAAAAAAAAbGFsYWxpDgAAAAAAAABuZXdQcm9ncmFtTmFtZQYAAAAAAAAAbm9ib2R5AQEAAAAAAAAABgAAAAAAAABfbXVzaWMFAAAAAAAAAAAAAAABAAAAAgAAAAMAAAAEAAAA'

In [21]:
import base64

In [22]:
base64.encodebytes(ffi.string(myAuth.app.id))

b'bmV3UHJvZ0lkX3VuaXF1ZV90aGluZw==\n'

In [23]:
base64.b64encode(b'newProgId_unique_thing').strip(b'=')

b'bmV3UHJvZ0lkX3VuaXF1ZV90aGluZw'

In [24]:
schemeName=b'safe-'+base64.b64encode(b'newProgId_unique_thing').strip(b'=')

In [25]:
schemeName

b'safe-bmV3UHJvZ0lkX3VuaXF1ZV90aGluZw'

In [26]:

sysUri = FFI()
with open('extracted_headers/safe_c_ffi_data.h','r') as f:
    myHeaders=f.read()
sysUri.cdef(myHeaders)
with open('extracted_headers/system_uri.h','r') as f:
    funs=f.read()
sysUri.cdef(funs)
lib2=sysUri.dlopen('../compiled_binaries/libsystem_uri.so')

In [27]:
@sysUri.callback("void(void *, FfiResult *)")
def installerFeedback(userData, result):
    print(userData)
    print(result.error_code)

```c
void install(char* bundle, 
             char* vendor, 
             char* name, 
             char* exec_args, 
             uint_least64_t exec_args_len, 
             char* icon, 
             char* schemes, 
             void* user_data, 
             void (*o_cb)(void* user_data, FfiResult* result));
```

In [28]:
print(ffi.string(myAuth.app.name))
print(ffi.string(myAuth.app.vendor))
print(ffi.string(myAuth.app.scope))

b'newProgramName'
b'nobody'
b'lalali'


In [29]:
import os
print(os.getcwd())

/home/yomuga/maidsafe/pySafe/pySafe


In [30]:
pathToHandler=(os.getcwd()+'/uriHandler.py').encode()

In [31]:
pathToHandler

b'/home/yomuga/maidsafe/pySafe/pySafe/uriHandler.py'

```c
myAuth,addData=AuthReq(newPermissions,1,1,id=b'newProgId',scope=b'lalali'
                       ,name=b'newProgramName',vendor=b'nobody',app_container=False)
```

In [32]:
bundle=sysUri.new('char[]',ffi.string(myAuth.app.id))
vendor=sysUri.new('char[]',ffi.string(myAuth.app.vendor))
name=sysUri.new('char[]',ffi.string(myAuth.app.name))
exec_args=sysUri.new('char[]',b'python')
exec_args2=sysUri.new('char[]',pathToHandler)
exec_args_len=2
icon=sysUri.new('char[]',b'')
schemes=sysUri.new('char[]',schemeName)
user_data=sysUri.NULL

In [33]:
exec_args_1 = sysUri.new('char*[]',[exec_args,exec_args2])

In [34]:
lib2.install(bundle,vendor,name,exec_args_1 ,exec_args_len,icon,schemes,user_data,installerFeedback)

<cdata 'void *' NULL>
0


```c 
void open_uri(char* uri, 
              void* user_data, 
              void (*o_cb)(void* user_data, FfiResult* result));
```

In [35]:
@sysUri.callback("void(void *, FfiResult *)")
def openerFeedback(userData, result):
    global res
    res = result
    print(userData)
    print(result.error_code)

In [36]:
ffi.string(myAuth.app.name)

b'newProgramName'

In [37]:
ffi.string(myAuth.app.vendor)

b'nobody'

In [38]:
ffi.string(myAuth.app.name)

b'newProgramName'

In [39]:
uri=sysUri.new('char[]',b'safe-auth://' + encodedAuth)
user_data=sysUri.NULL

In [40]:
lib2.open_uri(uri,user_data,openerFeedback)

<cdata 'void *' NULL>
0


In [41]:
def printCrustConfig(configAddress,configlength=42):
    lali=[]
    for i in range(configlength):
        lali.append(configAddress[i])
    print(lali)

```c 
void decode_ipc_msg(char* msg, 
                    void* user_data, 
                    void (*o_auth)(void* user_data, uint32_t req_id, AuthGranted* auth_granted), 
                    void (*o_unregistered)(void* user_data, uint32_t req_id, uint8_t* serialised_cfg, 
                                            uintptr_t serialised_cfg_len), 
                    void (*o_containers)(void* user_data, uint32_t req_id), 
                    void (*o_share_mdata)(void* user_data, uint32_t req_id), 
                    void (*o_revoked)(void* user_data), 
                    void (*o_err)(void* user_data, FfiResult* result, uint32_t req_id));
```

In [42]:
import time

In [43]:
import safeUtils

In [44]:
@ffi.callback("void(void *, uint32_t, AuthGranted *)")
def o_auth_decode_ipc_msg(userData, req_id, auth_granted):
    
    printCrustConfig(auth_granted.bootstrap_config)
    
    print('o_auth')
    print(userData)
    print(req_id)
    returnDict = ffi.from_handle(userData)
    #returnDict['AuthGranted'] = picData
    returnDict['AuthGranted'] = safeUtils.copy(auth_granted,ffi)
    #print(returnDict)
    myContainer=auth_granted.access_container_entry.containers[0]
    print(myContainer)
    print(auth_granted.access_container_entry.containers_len)
    print(auth_granted.access_container_entry.containers_cap)
    print(ffi.string(myContainer.name))
    print(myContainer.mdata_info)
    print(ffi.sizeof(myContainer.mdata_info))
    print(myContainer.permissions)
    print(ffi.sizeof(myContainer.permissions))


In [45]:
@ffi.callback("void(void *, uint32_t, uint8_t *, uintptr_t)")
def o_unregistered_decode_ipc_msg(userData, req_id, serialised_cfg, serialised_cfg_len):
    print('o_unregistered')
    print(req_id)

In [46]:
@ffi.callback("void(void *, uint32_t)")
def o_containers_decode_ipc_msg(userData, req_id):
    print('o_containers')
    print(req_id)

In [47]:
@ffi.callback("void(void *, uint32_t)")
def o_share_mdata_decode_ipc_msg(userData, req_id):
    print('o_share')
    print(req_id)

In [48]:
@ffi.callback("void(void *)")
def o_revoked_decode_ipc_msg(userData):
    print('o_revoked')
    print(req_id)

In [49]:
@ffi.callback("void(void *, FfiResult *, uint32_t)")
def o_error_decode_ipc_msg(userData, result, req_id):
    global res
    res = result
    print('o_err')
    print(userData)
    print(req_id)
    print(result.error_code)
    print(ffi.string(result.description))

In [50]:
with open('answer','r') as f:
    answer=f.readlines()

In [51]:
import re
data=re.compile("'safe-.+?:(.+?)'")

In [52]:
my=data.findall(answer[-1])

In [53]:
msg = ffi.new('char[]',my[-1].encode())

In [54]:
my[-1]

'AQAAAJg5ryQAAAAAAAAAACAAAAAAAAAAbhJf-9V6xLIwtiYS1eDRWmvSSRSBSUm2bmaPvH9TQoYgAAAAAAAAAAqM1x5AgFuxHRJYVDh2oY_eUVsniu50i2QJsszc4fMkIAAAAAAAAADe0Rxm7rghLJ-snOxS9plv8ZXZz9eg5agqrK7HFxM4u0AAAAAAAAAArtjGUW_n9nCRcUYIdiD2jCM2wc9C-xnnPhYyfatRCwje0Rxm7rghLJ-snOxS9plv8ZXZz9eg5agqrK7HFxM4uyAAAAAAAAAAFHijV65knaNjqPUztI0owutWYNkpO0lLYp7LzEuuGGQgAAAAAAAAAAWo4KdTl60iFXxXXUnT672Q27iB_CJtIC2HbOWbUIHNGQAAAAAAAAAQAAAAAAAAADE3OC42Mi43Ni44OjU0ODMTAAAAAAAAADEzOC42OC4xODUuMjE4OjU0ODMSAAAAAAAAADEzOC42OC4xODEuNTc6NTQ4MxIAAAAAAAAAMTM4LjY4LjE4MS42MDo1NDgzEgAAAAAAAAAxMzguNjguMTgxLjg2OjU0ODMSAAAAAAAAADEzOC42OC4xODEuODc6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4xNjg6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4xNzY6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4xNzk6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4xODA6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4xODI6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4yNDI6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4yNDM6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4yNDk6NTQ4MxIAAAAAAAAAMTM4LjY4LjE4OS4xNDo1NDgzEgAAAAAAAAAxMzguNjguMTg5LjE1OjU0ODMSAAAAAAAAADEzOC42OC4xODkuMTc6NTQ4MxIAAAA

In [55]:
returnDict={}

In [56]:
myHand=ffi.new_handle(returnDict)

In [57]:
lib.decode_ipc_msg(msg,myHand,o_auth_decode_ipc_msg,o_unregistered_decode_ipc_msg
                   ,o_containers_decode_ipc_msg,o_share_mdata_decode_ipc_msg,
                   o_revoked_decode_ipc_msg,o_error_decode_ipc_msg)

[25, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 49, 55, 56, 46, 54, 50, 46, 55, 54, 46, 56, 58, 53, 52, 56, 51, 19, 0, 0, 0, 0, 0, 0, 0, 49, 51]
o_auth
<cdata 'void *' 0x7f3efdd8c2c8>
615463320
<cdata 'ContainerInfo &' 0x55ed600c8980>
2
2
b'_music'
<cdata 'MDataInfo &' 0x55ed600c8988>
160
<cdata 'PermissionSet &' 0x55ed600c8a28>
5


#time.sleep(3)
lib.decode_ipc_msg(msg,ffi.new_handle(returnDict),o_auth_decode_ipc_msg,o_unregistered_decode_ipc_msg
                   ,o_containers_decode_ipc_msg,o_share_mdata_decode_ipc_msg,
                   o_revoked_decode_ipc_msg,o_error_decode_ipc_msg)

----

now the app

ffi.cdef('''
typedef struct { ...; } foo_t;
''')

In [58]:
@ffi.callback("void(void*)")
def disconnect_app_registered(user_data):
    print('disconnected - callback from app_registered')

In [59]:
innerData=None
@ffi.callback("void(void* , FfiResult* , App* app)")
def result_app_registered(user_data, result, app):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    returnDict = ffi.from_handle(user_data)
    returnDict['app']=app

lib.app_registered(myAuth.app.id,grantedAuth,ffi.NULL,disconnect_app_registered,result_app_registered)

In [60]:
lib.app_registered(myAuth.app.id,returnDict['AuthGranted'][0],ffi.new_handle(returnDict),disconnect_app_registered,result_app_registered)

results
error code: 
0


@ffi.callback("void(void* , FfiResult*)")
def result_clearCache(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

lib.app_reset_object_cache(returnDict['app'],ffi.new_handle(returnDict),result_clearCache)

In [61]:
containerName=ffi.new('unsigned char[]',b'_music')

In [65]:
@ffi.callback("void(void* , FfiResult*, MDataInfo *)")
def result_mdataInfoContainer(user_data, result, mdataInfo):
    global myMdata
    myMdata=mdataInfo
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(mdataInfo)
    #returnDict = ffi.from_handle(user_data)
    #returnDict['myMdata']=mdataInfo

In [66]:
lib.access_container_get_container_mdata_info(returnDict['app'],containerName,ffi.new_handle(returnDict),result_mdataInfoContainer)

results
error code: 
0
<cdata 'MDataInfo *' 0x7f3eefffc3d8>


In [76]:
returnDict['newMdata']=myMdata

createMutableData

@ffi.callback("void(void* , FfiResult*, MDataInfo *)")
def result_mdataInfoRand(user_data, result, mdataInfo):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(mdataInfo)
    returnDict = ffi.from_handle(user_data)
    returnDict['newMdata']=ffi.new('MDataInfo *',mdataInfo[0])

lib.mdata_info_random_public(15150,ffi.new_handle(returnDict),result_mdataInfoRand)

lali=returnDict['newMdata']

for i in range(22):
    print(lali.name[i])

@ffi.callback("void(void* , FfiResult*, MDataInfo *)")
def result_mdataInfoRand(user_data, result, mdataInfo):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(mdataInfo)
    returnDict = ffi.from_handle(user_data)
    returnDict['newMdata_private']=ffi.new('MDataInfo *',mdataInfo[0])

lib.mdata_info_random_private(15000,ffi.new_handle(returnDict),result_mdataInfoRand)

permission

In [101]:
@ffi.callback("void(void* , FfiResult*, unsigned long)")
def result_newPermissions(user_data, result, handle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(handle)

In [102]:
lib.mdata_permissions_new(returnDict['app'],ffi.NULL,result_newPermissions)

results
error code: 
0
1


In [79]:
@ffi.callback("void(void* , FfiResult* , unsigned long)")
def result_mdata_entries_new(user_data, result, entriesHandle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    #noway = ffi.from_handle(user_data)
    print(entriesHandle)
    #print(noway)
    #noway['mdata'] = entriesHandle

In [80]:
lib.mdata_entries_new(returnDict['app'],ffi.NULL,result_mdata_entries_new)

results
error code: 
0
1


In [81]:
oneKey=ffi.new('unsigned char[]',b'dask')

In [82]:
len(oneKey)

5

In [83]:
value=ffi.new('unsigned char[]',b'we get closer...')

In [84]:
len(value)

17

In [85]:
@ffi.callback("void(void* , FfiResult*)")
def result_mdata_entries_insert(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

In [87]:
lib.mdata_entries_insert(returnDict['app'],1,oneKey,len(oneKey),value,len(value),ffi.NULL,result_mdata_entries_insert)

results
error code: 
0


In [88]:
ffi.string(oneKey)

b'dask'

In [91]:
@ffi.callback("void(void* , FfiResult*, unsigned char*, unsigned long, unsigned long)")
def result_mdata_entries_get(user_data, result, value, val_length, noClueWhatThisIs):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(ffi.string(value))

In [92]:
lib.mdata_entries_get(returnDict['app'],1,oneKey,len(oneKey),ffi.NULL,result_mdata_entries_get)

results
error code: 
0
b'we get closer...'


In [109]:
@ffi.callback("void(void* , FfiResult* , unsigned long)")
def result_app_pub_sign_key(user_data, result, entriesHandle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(entriesHandle)

In [110]:
lib.app_pub_sign_key(returnDict['app'],ffi.NULL,result_app_pub_sign_key)

results
error code: 
0
3


In [111]:
@ffi.callback("void(void *, FfiResult *)")
def result_mdata_put(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

In [112]:
lib.mdata_put(returnDict['app'],returnDict['newMdata'],1,2,ffi.NULL,result_mdata_put)

results
error code: 
0


In [71]:
newPermissions=PermissionSet()

In [72]:
newPermissions.insert

True

In [73]:
@ffi.callback("void(void* , FfiResult*)")
def result_setPermissions(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    #returnDict = ffi.from_handle(user_data)
    #returnDict['newMdata']=ffi.new('PermissionSet *',handle[0])

In [74]:
lib.mdata_set_user_permissions(returnDict['app'],returnDict['newMdata_private'],
                               3,newPermissions,1,ffi.NULL,result_setPermissions)

results
error code: 
-100
b'Core error: Routing client error -> Access denied'


In [77]:
@ffi.callback("void(void* , FfiResult* , unsigned long, unsigned long)")
def result_sign_generate_key_pair(user_data, result, pubKeyHandle, secKeyHandle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(pubKeyHandle)
    print(secKeyHandle)

In [78]:
lib.sign_generate_key_pair(returnDict['app'],ffi.NULL,result_sign_generate_key_pair)

results
error code: 
0
4
5


In [75]:
@ffi.callback("void(void* , FfiResult*, PermissionSet*)")
def result_getPermissions(user_data, result, handle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(handle)
    #returnDict = ffi.from_handle(user_data)
    #returnDict['newMdata']=ffi.new('PermissionSet *',handle[0])

In [77]:
lib.mdata_permissions_get(returnDict['app'],1,4,ffi.new_handle(returnDict),result_getPermissions)

results
error code: 
-1011
b'Invalid sign public key handle'
<cdata 'PermissionSet *' NULL>


In [75]:
@ffi.callback("void(void *, FfiResult *)")
def result_mdata_put(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

In [76]:
lib.mdata_put(returnDict['app'],returnDict['newMdata'],1,2,ffi.NULL,result_mdata_put)

results
error code: 
-100
b'Core error: Routing client error -> Access denied'


In [105]:
@ffi.callback("void(void* , FfiResult*, PermissionSet*)")
def result_getPermissions(user_data, result, handle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(handle)
    returnDict = ffi.from_handle(user_data)
    returnDict['newMdata']=ffi.new('PermissionSet *',handle[0])

In [ ]:
lib.mdata_permissions_get(returnDict['app'],3,0,ffi.new_handle(returnDict),result_getPermissions)

results
error code: 
-1011
b'Invalid sign public key handle'
<cdata 'PermissionSet *' NULL>


permissionSet

In [62]:
@ffi.callback("void(void* , FfiResult*, unsigned long)")
def result_clearCache(user_data, result, handle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(handle)

In [63]:
lib.mdata_permissions_new(returnDict['app'],ffi.NULL,result_clearCache)

results
error code: 
0
1


In [62]:
@ffi.callback("void(void* , FfiResult*, unsigned long)")
def result_clearCache(user_data, result, handle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(handle)

In [63]:
lib.idata_new_self_encryptor(returnDict['app'],ffi.NULL,result_clearCache)

results
error code: 
0
1


In [64]:
with open('picData.jpeg','rb') as f:
    picData = f.read()

In [67]:
writeData=ffi.new('unsigned char[]',picData)

In [68]:
ffi.sizeof(writeData)

113025

In [69]:
@ffi.callback("void(void* , FfiResult*)")
def result_writeData(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

In [71]:
lib.idata_write_to_self_encryptor(returnDict['app'],1,writeData,ffi.sizeof(writeData),ffi.NULL,result_writeData)

results
error code: 
0


In [72]:
@ffi.callback("void(void* , FfiResult*, unsigned long)")
def result_cipher(user_data, result, handle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(handle)

In [73]:
lib.cipher_opt_new_symmetric(returnDict['app'],ffi.NULL,result_cipher)

results
error code: 
0
2


In [74]:
@ffi.callback("void(void* , FfiResult*, XorNameArray*)")
def result_closeEncryptor(user_data, result,name):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    for i in range(32):
        try:
            print(name[i])
        except:
            pass

In [75]:
lib.idata_close_self_encryptor(returnDict['app'],1,2,ffi.NULL,result_closeEncryptor)

results
error code: 
-16
b'Self-encryption error: Storage error: Routing client error -> Access denied'


In [59]:
lalai={}

In [61]:
@ffi.callback("void(void* , FfiResult* , unsigned long)")
def result_mdata_entries_new(user_data, result, entriesHandle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    noway = ffi.from_handle(user_data)
    print(entriesHandle)
    print(noway)
    #noway['mdata'] = entriesHandle

lib.mdata_entries_new(returnDict['app'],ffi.new_handle(returnDict),result_mdata_entries_new)

In [ ]:
lib.mdata_entries_new(returnDict['app'],ffi.new_handle(returnDict),result_mdata_entries_new)

results
error code: 
0


In [58]:
@ffi.callback("void(void* , FfiResult*)")
def result_access_container_refresh_access_info(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    #noway['mdata'] = entriesHandle

In [59]:
lib.access_container_refresh_access_info(returnDict['app'],ffi.NULL,result_access_container_refresh_access_info)

results
error code: 
-106
b'Core error: Routing client error -> Requested entry not found'


In [71]:
@ffi.callback("void(void* , FfiResult* , ContainerPermissions*, unsigned long)")
def result_access_container_fetch(user_data, result, permissions, entriesHandle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    noway = ffi.from_handle(user_data)
    print(entriesHandle)
    print(noway)
    #noway['mdata'] = entriesHandle

In [72]:
lib.access_container_fetch(returnDict['app'],ffi.new_handle(returnDict),result_access_container_fetch)

results
error code: 
-106
b'Core error: Routing client error -> Requested entry not found'


In [115]:
@ffi.callback("void(void* , FfiResult* , AccountInfo*)")
def result_app_account_info(user_data, result, info):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    #returnDict = ffi.from_handle(user_data)
    #if info != ffi.NULL:
    #    returnDict['info'] = safeUtils.copy(info,ffi)
    print(info)

In [116]:
lib.app_account_info(returnDict['app'],ffi.new_handle(returnDict),result_app_account_info)

results
error code: 
-100
b'Core error: Routing client error -> Access denied'
<cdata 'AccountInfo *' NULL>


In [121]:
@ffi.callback("void(void *, FfiResult *, ContainerPermissions *, unsigned long)")
def result_access_container_fetch(user_data, result, permissions, permissionLength):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    #returnDict = ffi.from_handle(user_data)
    if permissions != ffi.NULL:
        returnDict['info'] = safeUtils.copy(permissions,ffi)
    print(permissions[0])

In [122]:
lib.access_container_fetch(returnDict['app'],ffi.new_handle(returnDict),result_access_container_fetch)

results
error code: 
0
<cdata 'ContainerPermissions &' 0x7fbc04015230>


In [127]:
myConPerm=returnDict['info'][0]

In [129]:
ffi.string(myConPerm.cont_name)

b'_music'

In [ ]:
lib.access_container_get_container_mdata_info()

In [62]:
@ffi.callback("void(void *, FfiResult *)")
def result_app_init_logging(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    returnDict = ffi.from_handle(user_data)

In [63]:
output_file_name=ffi.new('char[]',b'/home/yomuga/maidsafe/pySafe/pySafe/log_out')

In [64]:
lib.app_init_logging(output_file_name,ffi.new_handle(returnDict),result_app_init_logging)

results
error code: 
0


In [128]:
@ffi.callback("void(void *, FfiResult *, char *)")
def result_app_output_log_path(user_data, result,logFile):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(ffi.string(logFile))
    returnDict['filePath'] = ffi.string(logFile)

In [131]:
file_name=ffi.new('char[]',b'log_out2')

In [132]:
lib.app_output_log_path(file_name,ffi.new_handle(returnDict),result_app_output_log_path)

results
error code: 
0
b'../compiled_binaries/log_out2'


In [73]:
newMdata=ffi.new('MDataInfo *')

In [74]:
newMdata.type_tag=15017

In [67]:
for i in range(32):
    newMdata.name[i]=i+1

In [68]:
@ffi.callback("void(void *, FfiResult *)")
def result_mdata_put(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

In [69]:
lib.mdata_put(returnDict['app'],newMdata,0,0,ffi.new_handle(returnDict),result_mdata_put)

results
error code: 
-100
b'Core error: Routing client error -> Access denied'


In [70]:
returnDict['app']

<cdata 'void * *' 0x5642362969c0>